In [ ]:
!nvidia-smi

Mon May  1 14:19:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/BarbosaRT/open_musiclm_colab.git

Cloning into 'open_musiclm_colab'...
remote: Enumerating objects: 1330, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 1330 (delta 70), reused 48 (delta 27), pack-reused 1209
Receiving objects: 100% (1330/1330), 1.84 MiB | 4.02 MiB/s, done.
Resolving deltas: 100% (918/918), done.


In [ ]:
!pip install -r open_musiclm_colab/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 101.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.0/708.0 kB 70.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00
  Created wheel for encodec: filename=encodec-0.1.1-py3-none-any.whl size=45775 sha256=4f04e1307718bbaab05d2f485846013001a6f5e40793ed15a19716208958cb61
  Stored in directory: /r

In [ ]:
#@title Install the trained models
!gdown 13pooxHbHk-TGecvGKB6U25o6gXrJ8qOt
!gdown 1brQas_CLpcWTaMOMe555SM4X98YjwjEQ
!gdown 12FGXgbUbICrE3CGteMbH-khiO5ZOI7oi
!gdown 1ZoGb9BBJiYVJXU1GCUMO-hMjetlvHsIe
!gdown 1fnAE5J_h674Co8oskAm66KYGR0RO7cdn
!gdown 1ic_rpXmjeVRTkGo2g91A0r9AEiDeEBlF
!mkdir results

Downloading...
From: https://drive.google.com/uc?id=13pooxHbHk-TGecvGKB6U25o6gXrJ8qOt
To: /content/clap.rvq.950_no_fusion.pt
100% 50.4M/50.4M [00:00<00:00, 59.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1brQas_CLpcWTaMOMe555SM4X98YjwjEQ
To: /content/coarse.transformer.18000.pt
100% 1.16G/1.16G [00:10<00:00, 106MB/s]
Downloading...
From: https://drive.google.com/uc?id=12FGXgbUbICrE3CGteMbH-khiO5ZOI7oi
To: /content/fine.transformer.24000.pt
100% 1.19G/1.19G [00:11<00:00, 99.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZoGb9BBJiYVJXU1GCUMO-hMjetlvHsIe
To: /content/kmeans_10s_no_fusion.joblib
100% 3.15M/3.15M [00:00<00:00, 257MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fnAE5J_h674Co8oskAm66KYGR0RO7cdn
To: /content/musiclm_large_small_context.json
100% 1.64k/1.64k [00:00<00:00, 10.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ic_rpXmjeVRTkGo2g91A0r9AEiDeEBlF
To: /content/semantic.transformer.14000.pt
100% 1.13G/1.13G [00:10<00:00, 1

In [ ]:
#@title Run MusicLM Infer
import os
import sys

import torch
import torchaudio
from einops import rearrange

from open_musiclm_colab.open_musiclm.config import load_model_config, create_musiclm_from_config

prompt = "The main soundtrack of an arcade game. It is fast-paced and upbeat, with a catchy electric guitar riff. The music is repetitive and easy to remember, but with unexpected sounds, like cymbal crashes or drum rolls."#@param {type:"string"}

prompts = [
    [str(prompt)],
]

if __name__ == '__main__':
  model_config = load_model_config('./musiclm_large_small_context.json')

  semantic_path = './semantic.transformer.14000.pt'
  coarse_path = './coarse.transformer.18000.pt'
  fine_path = './fine.transformer.24000.pt'
  return_coarse_wave = False
  kmeans_path = './kmeans_10s_no_fusion.joblib'
  rvq_path = './clap.rvq.950_no_fusion.pt'
  seed = 69420
  results_folder = './results'

  duration = 10

  print(f'semantic checkpoint {semantic_path}, coarse checkpoint {coarse_path}, fine checkpoint {fine_path}')
  print(f'kmeans path {kmeans_path}, rvq path {rvq_path}, return_coarse_wave {return_coarse_wave}')

  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  musiclm = create_musiclm_from_config(
  model_config=model_config,
  semantic_path=semantic_path,
  coarse_path=coarse_path,
  fine_path=fine_path,
  rvq_path=rvq_path,
  kmeans_path=kmeans_path,
  device=device)

  torch.manual_seed(seed)

  print('generating...')

  for prompt in prompts:
    generated_wave = musiclm.forward(
        text=prompt,
        output_seconds=duration,
        semantic_window_seconds=model_config.global_cfg.semantic_audio_length_seconds,
        coarse_window_seconds=model_config.global_cfg.coarse_audio_length_seconds,
        fine_window_seconds=model_config.global_cfg.fine_audio_length_seconds,
        semantic_steps_per_second=model_config.hubert_kmeans_cfg.output_hz,
        acoustic_steps_per_second=model_config.encodec_cfg.output_hz,
        return_coarse_generated_wave=return_coarse_wave,
    ).detach().cpu()

    print(generated_wave.shape)

    generated_wave = rearrange(generated_wave, 'b n -> b 1 n')
    for i, wave in enumerate(generated_wave):
        torchaudio.save(f'results/{prompt[i][:25]}_generated.wav', wave, musiclm.neural_codec.sample_rate)

semantic checkpoint ./semantic.transformer.14000.pt, coarse checkpoint ./coarse.transformer.18000.pt, fine checkpoint ./fine.transformer.24000.pt
kmeans path ./kmeans_10s_no_fusion.joblib, rvq path ./clap.rvq.950_no_fusion.pt, return_coarse_wave False


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.10/dist-packages/torchaudio/transforms/_transforms.py:611: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


You are using a model of type mert_model to instantiate a model of type hubert. This is not supported for all configurations of models and can yield errors.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MiniBatchKMeans from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 105MB/s]


generating...


generating predicted tokens: 100%|██████████| 150/150 [04:12<00:00,  1.68s/it]


torch.Size([1, 240000])


In [ ]:
#@title Run MusicLM top-match
import os
import sys

import torch
import torchaudio
from einops import rearrange
from pathlib import Path

from open_musiclm_colab.open_musiclm.config import load_model_config, create_musiclm_from_config

prompt = "The main soundtrack of an arcade game. It is fast-paced and upbeat, with a catchy electric guitar riff. The music is repetitive and easy to remember, but with unexpected sounds, like cymbal crashes or drum rolls."#@param {type:"string"}
duration = "10" #@param {type:"string"}
duration = int(duration)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_config = load_model_config('./musiclm_large_small_context.json')

semantic_path = './semantic.transformer.14000.pt'
coarse_path = './coarse.transformer.18000.pt'
fine_path = './fine.transformer.24000.pt'
return_coarse_wave = False
kmeans_path = './kmeans_10s_no_fusion.joblib'
rvq_path = './clap.rvq.950_no_fusion.pt'
seed = 69420
results_folder = './results'

num_samples = 1
num_top_matches = 1

musiclm = create_musiclm_from_config(
    model_config=model_config,
    semantic_path=semantic_path,
    coarse_path=coarse_path,
    fine_path=fine_path,
    rvq_path=rvq_path,
    kmeans_path=kmeans_path,
    device=device)

torch.manual_seed(seed)

print(f'prompt: {prompt}')

generated_wave, similarities = musiclm.generate_top_match(
    text=prompt,
    num_samples=num_samples,
    num_top_matches=num_top_matches,
    output_seconds=duration,
    semantic_window_seconds=model_config.global_cfg.semantic_audio_length_seconds,
    coarse_window_seconds=model_config.global_cfg.coarse_audio_length_seconds,
    fine_window_seconds=model_config.global_cfg.fine_audio_length_seconds,
    semantic_steps_per_second=model_config.hubert_kmeans_cfg.output_hz,
    acoustic_steps_per_second=model_config.encodec_cfg.output_hz,
    return_coarse_generated_wave=return_coarse_wave,
)

for i, (wave, sim) in enumerate(zip(generated_wave, similarities)):
    wave = rearrange(wave, 'b n -> b 1 n').detach().cpu()
    print(f'topk similarities: {sim}')
    for j, w in enumerate(wave):
        torchaudio.save(Path(results_folder) / Path(f'{prompt}_top_match_{j}.wav'), w, musiclm.neural_codec.sample_rate)